In [58]:
import pandas as pd
import nltk
import unicodedata
import numpy as np
import re        
# from nltk.stem.snowball import SnowballStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk.stem as stemmer
from nltk import sent_tokenize, word_tokenize
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer


ERROR: Could not find a version that satisfies the requirement Stemmer (from versions: none)
ERROR: No matching distribution found for Stemmer
You should consider upgrading via the '/home/abitporu/.pyenv/versions/3.9.0/envs/lede/bin/python3.9 -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'Stemmer'

In [60]:
# pd.options.display.max_colwidth = 100


In [34]:
def preprocess(text):
    new_text = text.lower()
    new_text = re.sub(r'http\S+', '', new_text)
    new_text = re.sub(r'www.\S+', '', new_text)
    new_text = re.sub("@[A-Za-z0-9_]+","", new_text)
    new_text = unicodedata.normalize('NFKD', new_text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    new_text = re.sub(r'[^\w\s]', '', new_text) # remove punc.
    new_text = re.sub(r'\d+','',new_text)# remove numbers
    return new_text

In [6]:
base = pd.read_csv("../dados/nlp/tweets_cleaned.csv")


In [6]:
# Sort by total of RTs
pd.set_option('display.max_colwidth', -1)
base.nlargest(50, columns='retweets_count')[['tweet','retweets_count']].head()

/tmp/ipykernel_31681/4038953154.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,tweet,retweets_count
3455,"acontecimentos como o de hoje mostram, para quem insiste em não ver, que o tráfico de drogas e o crime organizado vão muito além da favela. disparar tiros a esmo, do alto de um helicóptero, contra a favela não tem nada de combate ao crime! é apenas crueldade, faxina social mesmo.",5357.0
1734,"essas crianças uniformizadas, dentro da escola, abaixadinhas com medo de tomar tiro de helicóptero devia ser uma dessas cenas que causa demissão de alto a baixo na cadeia de comando. devia virar escândalo, não feito político.",3630.0
277,morador da favela da cidade de deus flagra uma granada sendo jogada em solo do helicóptero águia da polícia do rio! pic.twitter.com/zu9dlpiq1z,2065.0
9366,professor: - o mandante - comeu a inspetora - além de ser um sapão é inteligente pra caralho - as vezes começa a tremer e parece q vai explodir mas logo passa - já se mijou - foge tão bem da policia tal qual eu da diretora quando peço pra tomar agua e fico caminhando pic.twitter.com/2ks8tifyxl,1967.0
4553,atenção! um helicóptero da polícia está sobrevoando agora a maré e causando um terror em moradores e moradoras. muitos tiros são ouvidos e recebemos relatos de que moradores estão sendo impedidos entrar na favela! isso tudo em horário escolar! atirar a esmo é crime! é terrorismo! pic.twitter.com/z0wiwqox4m,1681.0


In [32]:
base['msg'] = base['tweet'].apply(preprocess)

In [33]:
base

,Unnamed: 0,timezone,tweet,hashtags,photos,retweets_count,replies_count,video,place,msg
0,0,-3.0,"em julho, consegui, depois de um ano de pedido...",[],[],2.0,1.0,0.0,NaN,em julho consegui depois de um ano de pedidos ...
1,1,-3.0,esse caveirao aéreo praticamente na minha jane...,[],[],0.0,0.0,0.0,NaN,esse caveirao aereo praticamente na minha jane...
2,2,-3.0,#bdrj para esses bandidos no alto do morro mer...,['#bdrj'],[],0.0,0.0,0.0,NaN,bdrj para esses bandidos no alto do morro mere...
3,3,-3.0,patrulheiro de assalto aéreo picareta squeak p...,[],[],0.0,0.0,0.0,NaN,patrulheiro de assalto aereo picareta squeak p...
4,4,-3.0,"eu acho que se eu pago um festival, chego lá e...",[],[],0.0,0.0,0.0,NaN,eu acho que se eu pago um festival chego la e ...
...,...,...,...,...,...,...,...,...,...,...
10914,10914,-3.0,será que só eu que não ouvir águia e nem tiro ??,[],[],2.0,0.0,0.0,NaN,sera que so eu que nao ouvir aguia e nem tiro
10915,10915,-3.0,"tiros, águia uma hora dessa 😴🖓",[],[],1.0,0.0,0.0,NaN,tiros aguia uma hora dessa
10916,10916,-3.0,se o aguia vim de troia querendo prejudicar os...,[],[],0.0,0.0,0.0,NaN,se o aguia vim de troia querendo prejudicar os...
10917,10917,-3.0,e se o águia vim de troia vim prejudicar os am...,[],[],0.0,0.0,0.0,NaN,e se o aguia vim de troia vim prejudicar os am...


In [35]:
snowball = SnowballStemmer(language = 'portuguese')
def stemming(words):
    new = []
    stem_words = [snowball.stem(x) for x in (words[:])]
    new.append(stem_words)
    return new

stemming(base.msg)

[['em julho consegui depois de um ano de pedidos e recursos via lei de acesso a informacao publicar dados sobre uso do helicoptero da policia civil o caveirao aereo em operacoes em favelas   ',
  'esse caveirao aereo praticamente na minha janela  eu hein acorda assim',
  'bdrj para esses bandidos no alto do morro merecem um caveirao aereo helicoptero mandando bala neles e nos amiguinhos ah mas vcs defendem bandidos ne esqueci que nao gostam do helicoptero nas operaco',
  'patrulheiro de assalto aereo picareta squeak picareta flamingo rosa emote andando e assobiando nog ops honor guard rogue spider knight frozen glow caveirao emote cabare e country rubi infiltrador anjo rodeio de foguetes essa e minha listan acaba por aq',
  'eu acho que se eu pago um festival chego la e tem um palco favela que imita barulho de caveirao aereo nao tem dinheiro gasto certo eu meto o pe na hor',
  'e voce que gasta  milhoes em um caveirao aereo pra metralhar pessoas inocentes so pela cor de sua pele e o ba

In [49]:
### STOPWORDS

stopwords = nltk.corpus.stopwords.words('portuguese')

stopwords_en = nltk.corpus.stopwords.words('english')

newStopWords = ['lula','http','3r6nsl','u200d','helicoptero','policia',
                'aeronave','ly','tb','tche', 'pra', '…','cv43sn','gate21','xa0','tinyurl',
                'helicóptero','durante','aguia','bit','https','fu','ahh','br','tiro',
                'bit','favela','sido','cara','cv43sn\'','enquanto', 'pic', 'twitter', 'www', 'ser'
               'ver','suzano','tipo','xa0\'','bom','bolivia','anti','diz','xa0helicoptero','apos','dois',
                'list','rio','da','em','quem','por','na','mas','ja','era','para','mais','se','nao','do',
                'que','de','ou','com','teria','sempre','outro','ao','os','duas','at','uma','um',
                'tiros','tiro','disparo','helicoptero','q','ta','pq','html','01','2011','tá','ig_twitter_share','uns']

stopwords.extend(newStopWords)

stopwords.extend(stopwords_en)

In [50]:
# Interface lemma tokenizer from nltk with sklearn
class LemmaTokenizer:
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`']
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if t not in self.ignore_tokens]

# Lemmatize the stop words
tokenizer=LemmaTokenizer()

In [51]:
tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words=stopwords,tokenizer=tokenizer)

doc_term_matrix = tfidf_vect.fit_transform(base.msg.values.astype('U'))

/home/abitporu/.pyenv/versions/3.9.0/envs/lede/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", "'d", "'ll", "'re", "'s", "'ve", 'could', 'doe', 'doi', 'es', 'forum', 'ha', 'might', 'must', "n't", 'need', 'sha', 'un', 'wa', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [54]:
pd.DataFrame(doc_term_matrix.toarray(), columns=tfidf_vect.get_feature_names()).columns[:30]

Index(['aaa', 'aaaa', 'aaaaa', 'aaaaaaaaaa', 'aaah', 'aah', 'ab', 'abafado',
       'abafados', 'abafar', 'abaixa', 'abaixadas', 'abaixadinhas', 'abaixado',
       'abaixava', 'abaixei', 'abaixo', 'abandonada', 'abandonado',
       'abastecer', 'abate', 'abatem', 'abaten', 'abatendo', 'abater',
       'abateram', 'abates', 'abateu', 'abatida', 'abatido'],
      dtype='object')

In [55]:
# Topic model w/ NMF

nmf = NMF(n_components=3, random_state=42)
nmf.fit(doc_term_matrix )

for i,topic in enumerate(nmf.components_):
    print(f'Top 10 words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-30:]])
    print('\n')

/home/abitporu/.pyenv/versions/3.9.0/envs/lede/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


Top 10 words for topic #0:
['gente', 'mandado', 'alto', 'voando', 'desse', 'dessa', 'tava', 'nada', 'porra', 'hora', 'caralho', 'morro', 'tao', 'estao', 'baixinho', 'crlh', 'acordei', 'mano', 'mane', 'rasante', 'mt', 'deus', 'passou', 'crl', 'passando', 'varios', 'p', 'cima', 'baixo', 'dando']


Top 10 words for topic #1:
['moradores', 'atirando', 'alemao', 'dia', 'deus', 'perto', 'n', 'deu', 'manha', 'governador', 'sao', 'hoje', 'complexo', 'rua', 'nada', 'ser', 'gente', 'pm', 'ai', 'escola', 'la', 'ate', 'agora', 'dar', 'cima', 'sobrevoando', 'operacao', 'vai', 'casa', 'aqui']


Top 10 words for topic #2:
['escutando', 'acordada', 'hoje', 'ouvindo', 'agr', 'acordo', 'escutei', 'sobrevoando', 'es', 'manha', 'nada', 'ouvi', 'dormir', 'susto', 'despertador', 'acordou', 'acorda', 'c', 'bala', 'morador', 'acerta', 'som', 'perdida', 'terror', 'botando', 'chega', 'acordar', 'dia', 'acordei', 'barulho']




In [43]:
LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(doc_term_matrix)

for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-30:]])
    print('\n')

LatentDirichletAllocation(n_components=5, random_state=42)